In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
import numpy as np
from PIL import Image

def func(video_path, threshold=50):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Get the frames per second (fps) of the video
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Initialize variables
    prev_frame = None
    key_frames = []
    frame_list = []

    while True:
        ret, frame = cap.read()

        if not ret:
            break

        # Convert the frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculate the absolute difference between the current and previous frames
        if prev_frame is not None:
            diff = cv2.absdiff(prev_frame, gray_frame)

            # Calculate the percentage of pixels that are different
            percentage_diff = np.sum(diff) / float(gray_frame.size)

            # If the percentage difference is above the threshold, consider it a key frame
            if percentage_diff > threshold:
                # Resize the frame to the target size
                resized_frame = cv2.resize(frame, (244,((int)((frame.shape[0]/frame.shape[1])*244))))
                key_frames.append((int(cap.get(cv2.CAP_PROP_POS_FRAMES)), resized_frame))
                frame_list.append(Image.fromarray(cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)))

        prev_frame = gray_frame

    # Release the video capture object
    cap.release()

    return key_frames, fps, frame_list

# Example usage
video_path = 'https://video.twimg.com/ext_tw_video/1030439106285846529/pu/vid/640x360/w5ayc4YSlQ1Of8dq.mp4?tag=5%27,%20bitrate=832000)],%20duration=25.04,%20views=80'
key_frames, fps, f_list = extract_key_frames(video_path)

# Print key frame numbers
for frame_number, _ in key_frames:
    print(f"Key Frame at Frame {frame_number}")

Key Frame at Frame 205
Key Frame at Frame 255
Key Frame at Frame 304


In [ ]:
# test url ='https://video.twimg.com/ext_tw_video/1030439106285846529/pu/vid/640x360/w5ayc4YSlQ1Of8dq.mp4?tag=5%27,%20bitrate=832000)],%20duration=25.04,%20views=80'

In [ ]:
import cv2
from PIL import Image
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to("cuda")

preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [ ]:
# Function to preprocess frames and concatenate them
def process_video(video_path, text, num_frames):
    frames = f_list

    # Concatenate frame features
    video_features = [processor(frame, text, return_tensors="pt").pixel_values.to("cuda") for frame in frames]
    video_input = torch.stack(video_features).mean(dim=0)  # You can adjust the aggregation method

    return video_input

# Example usage
video_path = 'https://video.twimg.com/ext_tw_video/1030439106285846529/pu/vid/640x360/w5ayc4YSlQ1Of8dq.mp4?tag=5%27,%20bitrate=832000)],%20duration=25.04,%20views=80'
num_frames = len(key_frames)  # Adjust based on your requirements
text ="Describe this in detail "

video_input = process_video(video_path, text, num_frames)

# Now you can use 'video_input' as input to your image-language model
out = model.generate(pixel_values=video_input)
caption = processor.decode(out[0], skip_special_tokens=True)
print(caption)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


a group of people are playing in the dirt


In [ ]:
# OLD CODE

# import cv2
# import numpy as np
# from PIL import Image

# def extract_key_frames(video_path, threshold=50):
#     # Open the video file
#     cap = cv2.VideoCapture(video_path)

#     # Get the frames per second (fps) of the video
#     fps = cap.get(cv2.CAP_PROP_FPS)

#     # Initialize variables
#     prev_frame = None
#     key_frames = []
#     frame_list = []

#     while True:
#         ret, frame = cap.read()

#         if not ret:
#             break

#         # Convert the frame to grayscale
#         gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#         # Calculate the absolute difference between the current and previous frames
#         if prev_frame is not None:
#             diff = cv2.absdiff(prev_frame, gray_frame)

#             # Calculate the percentage of pixels that are different
#             percentage_diff = np.sum(diff) / float(gray_frame.size)

#             # If the percentage difference is above the threshold, consider it a key frame
#             if percentage_diff > threshold:
#                 key_frames.append((int(cap.get(cv2.CAP_PROP_POS_FRAMES)), frame))
#                 frame_list.append(Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)))

#         prev_frame = gray_frame

#     # Release the video capture object
#     cap.release()

#     return key_frames, fps, frame_list

# # Example usage
# video_path = 'https://video.twimg.com/ext_tw_video/1030439106285846529/pu/vid/640x360/w5ayc4YSlQ1Of8dq.mp4?tag=5%27,%20bitrate=832000)],%20duration=25.04,%20views=80'
# key_frames, fps, f_list = extract_key_frames(video_path)

# # Print key frame numbers
# for frame_number, _ in key_frames:
#     print(f"Key Frame at Frame {frame_number}")


Key Frame at Frame 205
Key Frame at Frame 255
Key Frame at Frame 304
